<a href="https://colab.research.google.com/github/hfernandescfc/RaspandoCartola/blob/main/Carstats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import json

**Adding market database**

In [ ]:
url = 'https://api.cartolafc.globo.com/atletas/mercado'
r = requests.get(url, 'html.parser')

soup = r.json()

soup.keys()

dict_keys(['clubes', 'posicoes', 'status', 'atletas'])

**Adding players to a DataFrame**

In [ ]:
url = 'https://api.cartolafc.globo.com/atletas/mercado'
r = requests.get(url, 'html.parser')

soup = r.json()

atletas = pd.DataFrame(soup['atletas'])

atletas.head()

,scout,atleta_id,rodada_id,clube_id,posicao_id,status_id,pontos_num,preco_num,variacao_num,media_num,jogos_num,slug,apelido,apelido_abreviado,nome,foto
0,"{'A': 3, 'DS': 24, 'FC': 22, 'FD': 3, 'FF': 5,...",71937,19,327,4,6,0.0,8.29,0.00,4.22,13,juninho-valoura,Juninho Valoura,J. Valoura,Paulo Roberto Valoura Júnior,https://s.glbimg.com/es/sde/f/2021/06/04/bb9cd...
1,"{'DE': 24, 'FS': 4, 'GS': 9, 'SG': 3}",71631,19,275,1,7,0.0,15.01,0.00,4.00,8,weverton,Weverton,Weverton,Weverton Pereira da Silva,https://s.glbimg.com/es/sde/f/2021/04/30/33d63...
2,"{'A': 1, 'DS': 13, 'FC': 6, 'FF': 1, 'FS': 12,...",87225,19,293,3,7,4.3,7.84,0.44,1.94,14,pedro-henrique,Pedro Henrique,P. Henrique,Pedro Henrique Ribeiro Gonçalves,https://s.glbimg.com/es/sde/f/2021/05/02/f5dd1...
3,"{'A': 3, 'CA': 4, 'DS': 23, 'FC': 20, 'FD': 5,...",78435,19,262,5,6,0.0,7.52,0.00,3.73,14,vitinho,Vitinho,Vitinho,Victor Vinícius Coelho dos Santos,https://s.glbimg.com/es/sde/f/2021/05/01/4fafe...
4,"{'CV': 1, 'DS': 26, 'FC': 12, 'FD': 1, 'FS': 1...",79334,19,327,3,7,0.0,3.54,0.00,2.60,10,ricardo-silva,Ricardo Silva,R. Silva,Ricardo César Dantas da Silva,https://s.glbimg.com/es/sde/f/2021/06/04/a9554...


In [ ]:
atletas = pd.DataFrame(soup['atletas'])

As it can be seen, the scout columns contains dictionaries with the scouts performed by the player in the previous match. Our goal is to transform each possible scout in new columns and fill it with the obtained scout.

In [ ]:
#print(list(atletas['scout'][0].keys()))

scouts = []

for row in atletas['scout']:
  for sct in row.keys():
    if sct not in scouts:
      scouts.append(sct)
      

for scout in scouts:
  atletas[scout] = [x.get(scout) if scout in x else 0 for x in atletas['scout']]

atletas = atletas.drop(columns='scout')

In [ ]:
prox_rodada = atletas['rodada_id'][0]

#ult_rodada = prox_rodada - 1

atletas.head()


,atleta_id,rodada_id,clube_id,posicao_id,status_id,pontos_num,preco_num,variacao_num,media_num,jogos_num,slug,apelido,apelido_abreviado,nome,foto,A,DS,FC,FD,FF,FS,G,I,PI,DE,GS,SG,PC,CA,CV,FT,DP,GC,PS,PP
0,71937,19,327,4,6,0.0,8.29,0.00,4.22,13,juninho-valoura,Juninho Valoura,J. Valoura,Paulo Roberto Valoura Júnior,https://s.glbimg.com/es/sde/f/2021/06/04/bb9cd...,3,24,22,3,5,29,2,1,107,0,0,0,0,0,0,0,0,0,0,0
1,71631,19,275,1,7,0.0,15.01,0.00,4.00,8,weverton,Weverton,Weverton,Weverton Pereira da Silva,https://s.glbimg.com/es/sde/f/2021/04/30/33d63...,0,0,0,0,0,4,0,0,0,24,9,3,0,0,0,0,0,0,0,0
2,87225,19,293,3,7,4.3,7.84,0.44,1.94,14,pedro-henrique,Pedro Henrique,P. Henrique,Pedro Henrique Ribeiro Gonçalves,https://s.glbimg.com/es/sde/f/2021/05/02/f5dd1...,1,13,6,0,1,12,0,0,85,0,0,3,1,0,0,0,0,0,0,0
3,78435,19,262,5,6,0.0,7.52,0.00,3.73,14,vitinho,Vitinho,Vitinho,Victor Vinícius Coelho dos Santos,https://s.glbimg.com/es/sde/f/2021/05/01/4fafe...,3,23,20,5,15,16,2,0,97,0,0,0,0,4,0,0,0,0,0,0
4,79334,19,327,3,7,0.0,3.54,0.00,2.60,10,ricardo-silva,Ricardo Silva,R. Silva,Ricardo César Dantas da Silva,https://s.glbimg.com/es/sde/f/2021/06/04/a9554...,0,26,12,1,0,10,0,2,32,0,0,2,1,0,1,0,0,0,0,0


Now we need to add data about the matches in the round

In [ ]:
clubes = pd.DataFrame(soup['clubes'])

clubes = clubes.transpose()

clubes = clubes.rename(columns={'id':'clube_id','nome': 'equipe'})

clubes = clubes.drop(columns=['abreviacao', 'escudos', 'nome_fantasia'])

atletas = atletas.merge(clubes, how='left')

atletas.columns

clubes.head(20)


,clube_id,equipe
1371,1371,Cuiabá
262,262,Flamengo
264,264,Corinthians
265,265,Bahia
266,266,Fluminense
275,275,Palmeiras
276,276,São Paulo
277,277,Santos
280,280,Bragantino
282,282,Atlético-MG


In [ ]:
posicoes = pd.DataFrame(soup['posicoes'])

posicoes = posicoes.transpose()

#posicoes.head()

posicoes = posicoes.rename(columns={'id':'posicao_id', 'abreviacao': 'posicao'})

posicoes = posicoes.drop(columns='nome')

atletas = atletas.merge(posicoes, how='left')

atletas.head()


,atleta_id,rodada_id,clube_id,posicao_id,status_id,pontos_num,preco_num,variacao_num,media_num,jogos_num,slug,apelido,apelido_abreviado,nome,foto,A,DS,FC,FD,FF,FS,G,I,PI,DE,GS,SG,PC,CA,CV,FT,DP,GC,PS,PP,equipe,posicao
0,71937,19,327,4,6,0.0,8.29,0.00,4.22,13,juninho-valoura,Juninho Valoura,J. Valoura,Paulo Roberto Valoura Júnior,https://s.glbimg.com/es/sde/f/2021/06/04/bb9cd...,3,24,22,3,5,29,2,1,107,0,0,0,0,0,0,0,0,0,0,0,América-MG,mei
1,71631,19,275,1,7,0.0,15.01,0.00,4.00,8,weverton,Weverton,Weverton,Weverton Pereira da Silva,https://s.glbimg.com/es/sde/f/2021/04/30/33d63...,0,0,0,0,0,4,0,0,0,24,9,3,0,0,0,0,0,0,0,0,Palmeiras,gol
2,87225,19,293,3,7,4.3,7.84,0.44,1.94,14,pedro-henrique,Pedro Henrique,P. Henrique,Pedro Henrique Ribeiro Gonçalves,https://s.glbimg.com/es/sde/f/2021/05/02/f5dd1...,1,13,6,0,1,12,0,0,85,0,0,3,1,0,0,0,0,0,0,0,Athlético-PR,zag
3,78435,19,262,5,6,0.0,7.52,0.00,3.73,14,vitinho,Vitinho,Vitinho,Victor Vinícius Coelho dos Santos,https://s.glbimg.com/es/sde/f/2021/05/01/4fafe...,3,23,20,5,15,16,2,0,97,0,0,0,0,4,0,0,0,0,0,0,Flamengo,ata
4,79334,19,327,3,7,0.0,3.54,0.00,2.60,10,ricardo-silva,Ricardo Silva,R. Silva,Ricardo César Dantas da Silva,https://s.glbimg.com/es/sde/f/2021/06/04/a9554...,0,26,12,1,0,10,0,2,32,0,0,2,1,0,1,0,0,0,0,0,América-MG,zag


In [ ]:
next = prox_rodada + 1

url = 'https://api.cartolafc.globo.com/partidas/' + str(prox_rodada)
r = requests.get(url, 'html.parser')

soup = r.json()

soup.keys()

dict_keys(['clubes', 'partidas', 'rodada'])

In [ ]:
url2 = 'https://api.cartolafc.globo.com/partidas/' + str(next)
r2 = requests.get(url2, 'html.parser')

soup2 = r2.json()

In [ ]:
partidas = pd.DataFrame(soup['partidas'])


partidas = partidas.drop(columns=['partida_id', 'clube_casa_posicao', 'aproveitamento_mandante', 'aproveitamento_visitante', 'clube_visitante_posicao', 'partida_data',
                                  'timestamp', 'local', 'valida', 'placar_oficial_mandante', 'placar_oficial_visitante', 'status_transmissao_tr', 'inicio_cronometro_tr', 'status_cronometro_tr',
                                  'periodo_tr', 'transmissao'])

#partidas = partidas.rename(columns={'clube_casa_id':'clube_id'})

print(partidas)


   clube_casa_id  clube_visitante_id
0            276                 327
1            265                 356
2           1371                 277
3            354                 275
4            262                 373
5            293                 292
6            282                 284
7            264                 286
8            285                 280
9            315                 266


In [ ]:
partidas2 = pd.DataFrame(soup2['partidas'])


partidas2 = partidas2.drop(columns=['partida_id', 'clube_casa_posicao', 'aproveitamento_mandante', 'aproveitamento_visitante', 'clube_visitante_posicao', 'partida_data',
                                  'timestamp', 'local', 'valida', 'placar_oficial_mandante', 'placar_oficial_visitante', 'status_transmissao_tr', 'inicio_cronometro_tr', 'status_cronometro_tr',
                                  'periodo_tr', 'transmissao'])

#partidas = partidas.rename(columns={'clube_casa_id':'clube_id'})

print(partidas2)

   clube_casa_id  clube_visitante_id
0            327                 293
1            286                1371
2            280                 315
3            277                 265
4            284                 354
5            275                 262
6            356                 282
7            373                 264
8            266                 276
9            292                 285


In [ ]:
ids = pd.concat([partidas['clube_casa_id'], partidas['clube_visitante_id']], ignore_index=True)
                
ids = pd.DataFrame(ids)

ids = ids.rename(columns={0:'clube_id'})

adversario_ult_jogo = []

mando_de_campo_ult_jogo = []

def lista_adversarios(df1, col1, df2, col2, col3, lista, mando):
  for x in df1[col1]:
    if x in df2[col2].values:
      lista.append(df2.iloc[df2[df2[col2] == x].index[0], 1])
      mando.append(1)
    else:
      lista.append(df2.iloc[df2[df2[col3] == x].index[0], 0])
      mando.append(0)

lista_adversarios(ids, 'clube_id', partidas, 'clube_casa_id','clube_visitante_id' , adversario_ult_jogo, mando_de_campo_ult_jogo)

''' for x in ids['clube_id']:
  if x in partidas['clube_casa_id'].values:
    adversario_ult_jogo.append(partidas.iloc[partidas[partidas['clube_casa_id'] == x].index[0], 1])
    mando_de_campo_ult_jogo.append(1)
  else:
    adversario_ult_jogo.append(partidas.iloc[partidas[partidas['clube_visitante_id'] == x].index[0], 0])
    mando_de_campo_ult_jogo.append(0) '''


print(adversario_ult_jogo, mando_de_campo_ult_jogo)



[327, 356, 277, 275, 373, 292, 284, 286, 280, 266, 276, 265, 1371, 354, 262, 293, 282, 264, 285, 315] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
ids2 = pd.concat([partidas2['clube_casa_id'], partidas2['clube_visitante_id']], ignore_index=True)
                
ids2 = pd.DataFrame(ids2)

ids2 = ids2.rename(columns={0:'clube_id'})

adversario_prox_jogo = []

mando_de_campo_prox_jogo = []


lista_adversarios(ids2, 'clube_id', partidas2, 'clube_casa_id','clube_visitante_id' , adversario_prox_jogo, mando_de_campo_prox_jogo)


print(adversario_prox_jogo, mando_de_campo_prox_jogo)


[293, 1371, 315, 265, 354, 262, 282, 264, 276, 285, 327, 286, 280, 277, 284, 275, 356, 373, 266, 292] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
ids['adversario_ult_jogo'] = adversario_ult_jogo

ids['mando_de_campo_ult_jogo'] = mando_de_campo_ult_jogo

ids = ids.merge(clubes, how='left')

ids = ids.drop(columns='clube_id')

ids = ids.rename(columns={'adversario_ult_jogo':'clube_id'})

ids = ids.rename(columns={'equipe':'adversario_ult_jogo'})

ids 


,clube_id,mando_de_campo_ult_jogo,adversario_ult_jogo
0,327,1,São Paulo
1,356,1,Bahia
2,277,1,Cuiabá
3,275,1,Ceará
4,373,1,Flamengo
5,292,1,Athlético-PR
6,284,1,Atlético-MG
7,286,1,Corinthians
8,280,1,Internacional
9,266,1,Chapecoense


In [ ]:
ids2['adversario_prox_jogo'] = adversario_prox_jogo

ids2['mando_de_campo_prox_jogo'] = mando_de_campo_prox_jogo

ids2 = ids2.merge(clubes, how='left')

ids2 = ids2.drop(columns='clube_id')

ids2 = ids2.rename(columns={'adversario_prox_jogo':'clube_id'})

ids2 = ids2.rename(columns={'equipe':'adversario_prox_jogo'})

ids2 

,clube_id,mando_de_campo_prox_jogo,adversario_prox_jogo
0,293,1,América-MG
1,1371,1,Juventude
2,315,1,Bragantino
3,265,1,Santos
4,354,1,Grêmio
5,262,1,Palmeiras
6,282,1,Fortaleza
7,264,1,Atlético-GO
8,276,1,Fluminense
9,285,1,Sport


In [ ]:
full2 = atletas.merge(ids, how='left', on='clube_id')

full2 = full2.drop(columns=['clube_id', 'posicao_id','status_id','slug','apelido_abreviado','foto', 'nome'])

full2

,atleta_id,rodada_id,pontos_num,preco_num,variacao_num,media_num,jogos_num,apelido,A,DS,FC,FD,FF,FS,G,I,PI,DE,GS,SG,PC,CA,CV,FT,DP,GC,PS,PP,equipe,posicao,mando_de_campo_ult_jogo,adversario_ult_jogo
0,71937,19,0.0,8.29,0.00,4.22,13,Juninho Valoura,3,24,22,3,5,29,2,1,107,0,0,0,0,0,0,0,0,0,0,0,América-MG,mei,1,São Paulo
1,71631,19,0.0,15.01,0.00,4.00,8,Weverton,0,0,0,0,0,4,0,0,0,24,9,3,0,0,0,0,0,0,0,0,Palmeiras,gol,1,Ceará
2,87225,19,4.3,7.84,0.44,1.94,14,Pedro Henrique,1,13,6,0,1,12,0,0,85,0,0,3,1,0,0,0,0,0,0,0,Athlético-PR,zag,0,Sport
3,78435,19,0.0,7.52,0.00,3.73,14,Vitinho,3,23,20,5,15,16,2,0,97,0,0,0,0,4,0,0,0,0,0,0,Flamengo,ata,0,Atlético-GO
4,79334,19,0.0,3.54,0.00,2.60,10,Ricardo Silva,0,26,12,1,0,10,0,2,32,0,0,2,1,0,1,0,0,0,0,0,América-MG,zag,1,São Paulo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,106940,19,0.0,1.00,0.00,0.00,0,Nicolas,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Juventude,ata,1,Corinthians
796,112984,19,0.0,0.75,0.00,0.00,1,Werton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Flamengo,ata,0,Atlético-GO
797,107963,19,0.0,2.00,0.00,0.00,0,Gustavo Henrique,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bahia,zag,0,Fortaleza
798,105521,19,0.0,1.00,0.00,0.00,0,Luan Sales,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Flamengo,lat,0,Atlético-GO


In [ ]:
full = atletas.merge(ids2, how='left', on='clube_id')

full = full.drop(columns=['rodada_id', 'clube_id', 'posicao_id', 'status_id',
       'pontos_num', 'preco_num', 'variacao_num', 'media_num', 'jogos_num',
       'slug', 'apelido', 'apelido_abreviado', 'nome', 'foto', 'DE', 'FS',
       'GS', 'SG', 'A', 'DS', 'FC', 'FD', 'FF', 'PI', 'CA', 'FT', 'PC', 'I',
       'DP', 'G', 'CV', 'PS', 'PP', 'GC','posicao', 'equipe'])

full.columns

Index(['atleta_id', 'mando_de_campo_prox_jogo', 'adversario_prox_jogo'], dtype='object')

In [ ]:
rodada = full2['rodada_id'][0]

full2.to_csv('2021rodada' + str(rodada) +'.csv')

rodada_confrontos = full2['rodada_id'][0] + 1

full.to_csv('2021confrontos' + str(rodada_confrontos)+ '.csv')

In [ ]:
'rodada' + str(rodada) + '.csv'

'rodada19.csv'

In [ ]:
#full['posicao'].unique()
for sct in scouts:
  full2['avg_' + sct] = full2[sct]/full2['jogos_num']


ata = full2[full2['posicao'] == 'ata']
mei = full2[full2['posicao'] == 'mei']
lat = full2[full2['posicao'] == 'lat']
zag = full2[full2['posicao'] == 'zag']
gol = full2[full2['posicao'] == 'gol']

max_scts_ata = {}

min_scts_ata = {}

max_scts_mei = {}

min_scts_mei = {}

max_scts_lat = {}

min_scts_lat = {}

max_scts_zag = {}

min_scts_zag = {}


In [ ]:

''' full2['avg_finalizações'] = full2['avg_FF'] + full2['avg_FT'] + full2['avg_FD']
full2['avg_decisões'] = full2['avg_A'] + full2['avg_G'] '''



for sct in scouts:
  max_scts_ata[sct] = max(ata['avg_' + sct])

for sct in scouts:
  max_scts_mei[sct] = max(mei['avg_' + sct])

for sct in scouts:
  max_scts_lat[sct] = max(lat['avg_' + sct])

for sct in scouts:
  max_scts_zag[sct] = max(zag['avg_' + sct])

for sct in scouts:
  min_scts_ata[sct] = min(ata['avg_' + sct])
for sct in scouts:
  min_scts_mei[sct] = min(mei['avg_' + sct])

for sct in scouts:
  min_scts_lat[sct] = min(lat['avg_' + sct])

for sct in scouts:
  min_scts_zag[sct] = min(zag['avg_' + sct])


for sct in scouts:
  ata[sct + '_N'] = (ata['avg_' + sct] - min_scts_ata[sct])/(max_scts_ata[sct] - min_scts_ata[sct])

for sct in scouts:
  mei[sct + '_N'] = (mei['avg_' + sct] - mei[sct])/(max_scts_mei[sct] - min_scts_mei[sct])

for sct in scouts:
  lat[sct + '_N'] = (lat['avg_' + sct] - min_scts_lat[sct])/(max_scts_lat[sct] - min_scts_lat[sct])

for sct in scouts:
  zag[sct + '_N'] = (zag['avg_' + sct] - min_scts_zag[sct])/(max_scts_zag[sct] - min_scts_zag[sct])



#display(full2.head())

''' full2 = full2.drop(columns=['rodada_id', 'pontos_num', 'preco_num', 'variacao_num',
       'media_num', 'jogos_num', 'apelido', 'CA', 'DS', 'FC', 'FD', 'FF', 'FS',
       'FT', 'G', 'I', 'PI', 'SG', 'A', 'DE', 'GS', 'DP', 'CV', 'PP', 'PC',
       'PS', 'GC','equipe', 'posicao', 'mando_de_campo_ult_jogo',
       'adversario_ult_jogo'])
 '''


#full2.columns




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

" full2 = full2.drop(columns=['rodada_id', 'pontos_num', 'preco_num', 'variacao_num',\n       'media_num', 'jogos_num', 'apelido', 'CA', 'DS', 'FC', 'FD', 'FF', 'FS',\n       'FT', 'G', 'I', 'PI', 'SG', 'A', 'DE', 'GS', 'DP', 'CV', 'PP', 'PC',\n       'PS', 'GC','equipe', 'posicao', 'mando_de_campo_ult_jogo',\n       'adversario_ult_jogo'])\n "

In [ ]:
normed_df = pd.concat([ata, mei, lat, zag])

normed_df = normed_df.drop(columns=['rodada_id', 'pontos_num', 'preco_num', 'variacao_num',
       'media_num', 'jogos_num', 'apelido', 'A', 'DS', 'FC', 'FD', 'FF', 'FS',
       'PI', 'CA', 'I', 'PC', 'SG', 'DE', 'GS', 'FT', 'G', 'PS', 'CV', 'DP',
       'PP', 'GC', 'equipe', 'posicao', 'mando_de_campo_ult_jogo',
       'adversario_ult_jogo', 'avg_A', 'avg_DS', 'avg_FC', 'avg_FD', 'avg_FF',
       'avg_FS', 'avg_PI', 'avg_CA', 'avg_I', 'avg_PC', 'avg_SG', 'avg_DE',
       'avg_GS', 'avg_FT', 'avg_G', 'avg_PS', 'avg_CV', 'avg_DP', 'avg_PP',
       'avg_GC'])

normed_df.head()

,atleta_id,A_N,DS_N,FC_N,FD_N,FF_N,FS_N,G_N,I_N,PI_N,DE_N,GS_N,SG_N,PC_N,CA_N,CV_N,FT_N,DP_N,GC_N,PS_N,PP_N
3,78435,0.571429,0.328571,0.571429,0.178571,0.535714,0.190476,0.142857,0.000000,0.535390,NaN,NaN,NaN,0.0,0.285714,0.0,0.000000,NaN,NaN,0.0,0.0
8,92696,0.000000,0.066667,0.480000,0.133333,0.033333,0.122222,0.066667,0.086275,0.128788,NaN,NaN,NaN,0.0,0.066667,0.0,0.000000,NaN,NaN,0.0,0.0
10,80076,0.000000,0.141176,0.729412,0.088235,0.176471,0.264706,0.235294,0.380623,0.304545,NaN,NaN,NaN,0.0,0.294118,0.0,0.235294,NaN,NaN,0.0,0.0
25,68835,0.000000,0.000000,0.200000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.0,0.000000,0.0,0.000000,NaN,NaN,0.0,0.0
26,105909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.077273,NaN,NaN,NaN,0.0,0.000000,0.0,0.000000,NaN,NaN,0.0,0.0


In [ ]:
normed_df = normed_df.melt(id_vars='atleta_id', value_vars=['DE_N', 'FS_N', 'GS_N', 'SG_N', 'DS_N', 'A_N', 'CA_N',
       'FC_N', 'FD_N', 'FF_N', 'FT_N', 'G_N', 'I_N', 'PI_N', 'PS_N', 'CV_N',
       'PC_N', 'PP_N', 'DP_N', 'GC_N'], var_name='Normed_Scout')



In [ ]:
normed_df.to_csv('medias_normalizadas.csv')